In [1]:
%pip install -r tic-tac-toe/requirements.txt
%pip install -e tic-tac-toe/

Note: you may need to restart the kernel to use updated packages.
Obtaining file:///C:/Users/Spencer/Documents/CUBoulder/CSCI7000/Assignment1/tic-tac-toe
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for tictactoe (pyproject.toml): started
  Building editable for tictactoe (pyproject.toml): finished with status 'done'
  Created wheel for tictactoe: filename=tictactoe-0.0.1-0.editable-py3-none-any.whl size=2824 sha256=ff40b4e98f2ff73fc762ad2aabb18d8d62a656459ca08e5825e514ace5b6c268
  Stored in directory: C:\Users\Spencer\AppD

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from tictactoe.board import Board

# Tasks
- Create Data
    - Create Recursive function to get all board positions
    - Create Function to get best move
- Create Transformer
- Train Transformer
- Write article

## Data Work

### Generate Data

In [357]:
size = 3
boards = {}
def generate_data(board):
    b = Board()
    if np.array2string(board.flatten()) not in list(boards.keys()):
        out_boards = []
        for i,j in b.get_best_moves(board):
            nb = Board()
            nb.set_board(np.copy(board))
            nb.place(i,j)
            out_boards.append(nb.to_str())
        boards[np.array2string(board.flatten())] = out_boards

    for i, j in list(zip(*np.where(board==0))):
        nb = Board()
        nb.set_board(np.copy(board))
        nb.place(i,j)
        new_board = nb.board
        try:
            generate_data(new_board)
        except:
            print(f"i: {i} j: {j} board:{new_board}")
            raise
generate_data(np.array([[0,0,0],
                        [0,0,0],
                        [0,0,0]]))

i: 1 j: 0 board:[[ 0  1  1]
 [-1 -1  0]
 [-1 -1  1]]
i: 2 j: 2 board:[[ 0  1  1]
 [ 0 -1  0]
 [-1 -1  1]]
i: 1 j: 1 board:[[ 0  1  1]
 [ 0 -1  0]
 [-1 -1  0]]
i: 0 j: 1 board:[[ 0  1  1]
 [ 0  0  0]
 [-1 -1  0]]
i: 2 j: 0 board:[[ 0  0  1]
 [ 0  0  0]
 [-1 -1  0]]
i: 0 j: 2 board:[[ 0  0  1]
 [ 0  0  0]
 [ 0 -1  0]]
i: 2 j: 1 board:[[ 0  0  0]
 [ 0  0  0]
 [ 0 -1  0]]


KeyboardInterrupt: 

In [370]:
fixed_data = {k:v for k, v in data.items() if v != []}

#### Save Data

In [4]:
len(list(boards.keys()))

6046

In [5]:
ex_key = list(boards.keys())[0]
print(f"ex_key: {ex_key} \n ex_vals: {boards[ex_key]}")

ex_key: [0 0 0 0 0 0 0 0 0] 
 ex_vals: ['[-1  0  0  0  0  0  0  0  0]', '[ 0 -1  0  0  0  0  0  0  0]', '[ 0  0 -1  0  0  0  0  0  0]', '[ 0  0  0 -1  0  0  0  0  0]', '[ 0  0  0  0 -1  0  0  0  0]', '[ 0  0  0  0  0 -1  0  0  0]', '[ 0  0  0  0  0  0 -1  0  0]', '[ 0  0  0  0  0  0  0 -1  0]', '[ 0  0  0  0  0  0  0  0 -1]']


In [371]:
import json
with open('./data/data.json', 'w') as f:
    json.dump(fixed_data, f, indent=4)

#### Load Data

In [372]:
import json

with open('./data/data.json', 'r') as f:
    data = json.load(f)

In [373]:
len(list(data.keys()))

5920

#### Split the Data

In [374]:
import random

train_percent, val_percent, test_percent = 0.7, 0.2, 0.1

data_keys = list(data.keys())
n = len(data_keys)
num_train, num_val, num_test = int(train_percent*n), int(val_percent*n), int(test_percent*n)

select_train = random.sample(data_keys, num_train)
val_and_test = [value for value in data_keys if value not in select_train]

select_val = random.sample(val_and_test, num_val)
select_test = [value for value in val_and_test if value not in select_val]

In [375]:
def boards_to_format(key):
    key = key[1:-1]
    return [int(c) for c in key.split(' ') if c in ['-1', '0', '1']]

train_data = {int(i): (boards_to_format(k), [boards_to_format(v) for v in data[k]]) for i, k in enumerate(select_train)}
val_data = {int(i): (boards_to_format(k), [boards_to_format(v) for v in data[k]]) for i, k in enumerate(select_val)}
test_data = {int(i): (boards_to_format(k), [boards_to_format(v) for v in data[k]]) for i, k in enumerate(select_test)}

In [376]:
print(len(list(train_data.keys())))
print(len(list(val_data.keys())))
print(len(list(test_data.keys())))

4144
1184
592


#### Save the Data

In [377]:
with open('./data/train.json', 'w') as f:
    json.dump(train_data, f)
with open('./data/val.json', 'w') as f:
    json.dump(val_data, f)
with open('./data/test.json', 'w') as f:
    json.dump(test_data, f)

#### Translate Data into dataloaders

In [401]:
from torch.utils.data import Dataset
class TicTacDataset(Dataset):
    def __init__(self, path = './data/train.json'):
        with open(path, 'r') as f:
            self.data = json.load(f)
        self.len = len(list(self.data.keys()))
        
        # self.inputs, self.outputs = [], []
        # for ins, outs in data.items():
        #     for out in outs:
        #         self.inputs.append(self.boards_to_format(ins))
        #         self.outputs.append(self.boards_to_format(out))

    # def boards_to_format(self, key):
    #     key = key[1:-1]
    #     return [int(c) for c in key.split(' ') if c in ['-1', '0', '1']]
        
    def __len__(self,):
        return self.len
    
    def __getitem__(self, idx):
        ins, outs = self.data[str(idx)]
        return ins, outs

In [402]:
train_dataset = TicTacDataset(path='./data/train.json')
val_dataset = TicTacDataset(path='./data/val.json')
test_dataset = TicTacDataset(path='./data/test.json')

In [403]:
from torch.utils.data import DataLoader

In [425]:
def collate(batch):
    # print(batch)
    ins = [torch.tensor(b[0])+1 for b in batch]
    # outs = [torch.tensor(np.average(np.array(b[1])+1, axis=0)) for b in batch]
    outs = [torch.tensor(b[1][0])+1 for b in batch]
    # print(outs)



    # max_outs = max([len(o) for o in outs])
    # padded_outs = []
    # for o in outs:
    #   pad_boxes = max_outs - len(o)
    #   padded_out = (o) + [[1,1,1,1,1,1,1,1,1]] * pad_boxes
    #   padded_outs.append(torch.tensor(padded_out, dtype=torch.float32)+1)
    return torch.stack(ins), torch.stack(outs)


In [426]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True, collate_fn=collate)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True, collate_fn=collate)

In [427]:
out = next(iter(train_dataloader))

## Model Work

In [428]:
import torch
from torch import nn
import torch.nn.functional as F

torch.manual_seed(42)

# Set default device to CUDA if available, otherwise CPU
if torch.cuda.is_available():
    torch.set_default_device('cuda')
else:
    torch.set_default_device('cpu')

### Build Model

In [429]:
vocab_size = 3
num_embedding = 32

num_heads = 8

num_blocks = 4
block_size = 9

dropout = 0.0

In [430]:
class Head(nn.Module):
    """ Singular Head"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(num_embedding, head_size, bias=False)
        self.query = nn.Linear(num_embedding, head_size, bias=False)
        self.value = nn.Linear(num_embedding, head_size, bias=False)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape

        k = self.key(x) # (B,T,C)
        q = self.query(x) # (B,T,C)

        # Computer attention
        # C**0.5 is to make the numbers smaller so softmax doesn't do weird things
        wei = q @ k.transpose(-2, -1) / C**0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        # 
        V = self.value(x)
        out = wei @ V # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [431]:
class MultiHeadAttention(nn.Module):
    """Multiple Attention Heads"""
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for i in range(num_heads)])
        self.project = nn.Linear(num_embedding, num_embedding)                  # Projection layer for gettting back into the residual pathway
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.project(out)
        out = self.dropout(out)
        return out

In [432]:
class FeedFoward(nn.Module):
    """Single Layer"""

    def __init__(self, num_embedding):
        super().__init__()

        self.m = nn.Sequential(nn.Linear(num_embedding, 4 * num_embedding), # 4* because they did it in the paper
                               nn.ReLU(),
                               nn.Linear(4 * num_embedding, num_embedding), # Projection layer for gettting back into the residual pathway
                               nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.m(x)

In [433]:
class Block(nn.Module):
    """"""

    def __init__(self, num_embedding, num_head):
        super().__init__()

        head_size = num_embedding // num_head
        self.self_attn = MultiHeadAttention(num_head, head_size)
        self.feed_fwd = FeedFoward(num_embedding)

        self.lay_norm1 = nn.LayerNorm(num_embedding)
        self.lay_norm2 = nn.LayerNorm(num_embedding)

    def forward(self, x):
        x = x + self.self_attn(self.lay_norm1(x))
        x = x + self.feed_fwd(self.lay_norm2(x))
        return x


In [437]:
class TicTacToeModel(nn.Module):
    def __init__(self):
        super().__init__()

        # Embeddings
        self.token_embedding_table = nn.Embedding(vocab_size,num_embedding)
        self.position_embedding_table = nn.Embedding(block_size, num_embedding)

        # Attention
        self.attn_blocks = nn.Sequential(
            *[Block(num_embedding, num_head = 4) for i in range(num_blocks)],
            nn.LayerNorm(num_embedding),
        )

        self.lm_head = nn.Linear(num_embedding, vocab_size)

    def forward(self, inputs, labels=None):
        B, T = inputs.shape
        # print(inputs[0])
        # print(labels[0])

        token_embedding = self.token_embedding_table(inputs) # (B, T, C)
        position_embedding = self.position_embedding_table(torch.arange(T)) # (T, C)
        x = token_embedding + position_embedding # (B, T, C)

        x = self.attn_blocks(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if labels is None:
            loss = None
        else:
            # print(logits.shape)
            # print(labels.shape)
            
            # loss = F.cross_entropy(logits, labels) # need to add 
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1), ignore_index=-1)
        return logits, loss


## Train Model

In [438]:
lr = 1e-3
epochs = 30

model = TicTacToeModel()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

for epoch in range(epochs):
    for batched_x, batched_y in train_dataloader:
        logits, loss = model(batched_x, batched_y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        print(f"Training Loss: {loss.item()}")

Training Loss: 1.4243710041046143
Training Loss: 1.2248929738998413
Training Loss: 1.1514091491699219
Training Loss: 1.0253156423568726
Training Loss: 0.9188567399978638
Training Loss: 0.8241943120956421
Training Loss: 0.7607859969139099
Training Loss: 0.685420036315918
Training Loss: 0.624018669128418
Training Loss: 0.5655301809310913
Training Loss: 0.5146426558494568
Training Loss: 0.47928163409233093
Training Loss: 0.4522138833999634
Training Loss: 0.41116735339164734
Training Loss: 0.39550676941871643
Training Loss: 0.38081228733062744
Training Loss: 0.3689066767692566
Training Loss: 0.352457731962204
Training Loss: 0.34605658054351807
Training Loss: 0.3385683000087738
Training Loss: 0.3335327208042145
Training Loss: 0.32615941762924194
Training Loss: 0.31354615092277527
Training Loss: 0.3108419179916382
Training Loss: 0.32639262080192566
Training Loss: 0.3128700852394104
Training Loss: 0.301074355840683
Training Loss: 0.3137728273868561
Training Loss: 0.29779481887817383
Training 

In [86]:
out = next(iter(train_dataloader))

In [ ]:
# Training loop
for epoch in range(30):
    for xenc_batch, y_batch, lengths in dataloader:

        # Move batch to GPU
        xenc_batch = xenc_batch.to(device)
        y_batch = y_batch.to(device)

        # Forward pass
        logits = xenc_batch @ W  # Log-counts, shape: [batch_size, max_seq_len, alphabet_size]
 
        logits = logits.view(-1, logits.size(-1))  # Shape: [batch_size * max_seq_len, vocab_size]
        y_batch = y_batch.view(-1)  # Shape: [batch_size * max_seq_len]

        # Compute the loss using CrossEntropyLoss
        loss = F.cross_entropy(logits, y_batch, ignore_index=ctoi['.'])
        
        # Backward pass
        W.grad = None  # Set gradients to zero before backward pass
        loss.backward()

        # Update weights (gradient descent)
        with torch.no_grad():  # Don't track this update in the graph
            W -= 5 * W.grad  # Gradient update
        xloss.append(loss.item())
    
    print(f"Epoch {epoch}, Loss: {loss.item()}")